In [ ]:
### 스타벅스 지도 시각화
# 입지 관련 가설 세우기
# 1. 거주 인구가 많은 지역에 스타벅스 매장이 많이 입지해있을 것
# 2. 직장인이 많은 지역에 스타벅스 매장이 많이 입지해있을 것

In [ ]:
### 데이터 수집 (서울시 내)
# 1. 스타벅스들의 위치
# 2. 인구 통계 데이터 수집

In [2]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd

path = 'c:/webdriver/chromedriver.exe'
driver = webdriver.Chrome(path)
driver.get('https://www.starbucks.co.kr/store/store_map.do')

C:\Users\tjoeun709\AppData\Local\Temp\ipykernel_2696\2211235179.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


In [3]:
local_btn = 'header.loca_search > h3 > a'
driver.find_element_by_css_selector(local_btn).click()
time.sleep(2)

seoul_btn = 'ul.sido_arae_box > li:nth-child(1) > a'
driver.find_element_by_css_selector(seoul_btn).click()
time.sleep(2)

all_btn = 'ul.gugun_arae_box > li:nth-child(1) > a'
driver.find_element_by_css_selector(all_btn).click()
time.sleep(3)

C:\Users\tjoeun709\AppData\Local\Temp\ipykernel_2696\3630988392.py:2: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector(local_btn).click()
C:\Users\tjoeun709\AppData\Local\Temp\ipykernel_2696\3630988392.py:6: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector(seoul_btn).click()
C:\Users\tjoeun709\AppData\Local\Temp\ipykernel_2696\3630988392.py:10: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector(all_btn).click()


In [29]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
datas = soup.select('ul.quickSearchResultBoxSidoGugun > li.quickResultLstCon')

starbucks_seoul = []

for i in range(len(datas)) :
    name = datas[i]['data-name']
    lat = datas[i]['data-lat']
    lon = datas[i]['data-long']
    typ = datas[i].select('i')[0]['class'][0][4:]
    addr = str(datas[i].select('p')[0]).split('<br/>')[0].split('>')[1]
    tel = str(datas[i].select('p')[0]).split('<br/>')[1].split('<')[0]
    starbucks_seoul.append([name, lat, lon, typ, addr, tel])


starbucks_seoul_df = pd.DataFrame(starbucks_seoul, columns=['매장명','위도','경도','매장타입','주소','전화번호'])

starbucks_seoul_df.to_excel('data/5_starbucks_seoul.xlsx', index=False)


#### 서울 열린 데이터광장 공공데이터 수집 및 처리 

In [1]:
### 주민등록인구 통계 데이터
import pandas as pd

sgg_pop_df = pd.read_csv('data/seoul_popul.txt', sep='\t', header=2)
columns = {
    '기간' : 'quarter',
    '자치구' : 'area',
    '계' : 'total',
    '계.1' : 'local_total',
    '계.2' : 'foreign_total',
    '남자' : 'male_total',
    '남자.1' : 'local_male',
    '남자.2' : 'foreign_male',
    '여자' : 'female_total',
    '여자.1' : 'local_female',
    '여자.2' : 'foreign_female',
    '세대' : 'house',
    '세대당인구' : 'popul_per_house',
    '65세이상고령자' : 'aged'
}
sgg_pop_df.rename(columns = columns, inplace = True)
sgg_pop_df.head()

,quarter,area,house,total,male_total,female_total,local_total,local_male,local_female,foreign_total,foreign_male,foreign_female,popul_per_house,aged
0,2022.1/4,합계,"4,451,444","9,732,617","4,716,704","5,015,913","9,506,778","4,613,910","4,892,868","225,839","102,794","123,045",2,"1,628,980"
1,2022.1/4,종로구,"73,766","153,684","74,018","79,666","144,275","69,911","74,364","9,409","4,107","5,302",2,"28,064"
2,2022.1/4,중구,"63,767","131,450","63,869","67,581","122,173","59,448","62,725","9,277","4,421","4,856",2,"24,742"
3,2022.1/4,용산구,"111,207","236,518","114,571","121,947","222,169","106,735","115,434","14,349","7,836","6,513",2,"39,461"
4,2022.1/4,성동구,"134,533","291,609","141,662","149,947","285,073","138,847","146,226","6,536","2,815","3,721",2,"46,896"


In [14]:
### 불필요한 데이터 제거 (구, 구별 인구합계만 추리기)
sgg_pop_df_selected = sgg_pop_df[1:]

columns = ['area', 'total']
sgg_pop_df_final = sgg_pop_df_selected[columns]
sgg_pop_df_final.columns = ['시군구명','주민등록인구수']

sgg_pop_df_final.to_excel('data/sgg_pop.xlsx', index=False)


In [ ]:
### 서울시 동별 사업체현황 통계
sgg_biz_df = pd.read_csv('data/seoul_corp.txt', sep='\t', header=2)

columns = {
    '기간' : 'YEAR',
    '자치구' : 'AREA',
    '동' : 'DONG',
    '사업체수' : 'CORP_CNT',
    '여성대표자' : 'FEMALE_REP',
    '계' : 'EMP_TOTAL',
    '남' : 'EMP_MALE',
    '여' : 'EMP_FEMALE'
}
sgg_biz_df.rename(columns=columns, inplace=True)
sgg_biz_df.head()

In [ ]:
### 시군구별 사업체 현황 데이터 추출(구별-동별을 합계해서)
condition = sgg_biz_df['DONG'] == '소계'
sgg_biz_df_selected = sgg_biz_df[condition]

sgg_biz_df_selected.head()

### 구, 사업체 수, 종사자 수만 추리기
columns = ['AREA', 'EMP_TOTAL', 'CORP_CNT']
sgg_biz_df_final = sgg_biz_df_selected[columns]
sgg_biz_df_final.columns = ['시군구명','종사자수','사업체수']
sgg_biz_df_final.head()

sgg_biz_df_final.to_excel('data/sgg_biz.xlsx', index=False)

In [2]:
### 1. 서울시 인구 조사(시군구별) - data/sgg_pop.xlsx
### 2. 서울시 사업체데이터 수집 - data/sgg_biz.xlsx
### 3. 스타벅스 매장 목록 = data/5_starbucks_seoul.xlsx
import pandas as pd

In [3]:
seoul_starbucks = pd.read_excel('data/5_starbucks_seoul.xlsx')  # 스타벅스 데이터
sgg_pop = pd.read_excel('data/sgg_pop.xlsx')    # 인구데이터
sgg_biz = pd.read_excel('data/sgg_biz.xlsx')    # 사업체 통계데이터

In [ ]:
sgg_name = []
for address in seoul_starbucks['주소'] :
    sgg = address.split()[1]
    sgg_name.append(sgg)

seoul_starbucks['시군구명'] = sgg_name
seoul_starbucks.tail()
seoul_starbucks.to_excel('data/seoul_starbucks_list.xlsx', index=False)

#### 스타벅스 분석용 데이터 만들기 

In [ ]:
### 시군구 목록 데이터 수집
seoul_sgg = pd.read_excel('data/seoul_sgg_list.xlsx')
seoul_sgg.head()

In [ ]:
### 서울 스타벅스 매장 목록에서 시군구별 스타벅스 매장 수 카운트
starbucks_sgg_count = seoul_starbucks.pivot_table(values='매장명',index='시군구명', aggfunc='count').rename(columns={'매장명': '스타벅스_매장수'})
starbucks_sgg_count.head()

In [ ]:
### 서울시 시군구 목록데이터에 스타벅스 매장수 데이터 병합

seoul_sgg = pd.merge(seoul_sgg, starbucks_sgg_count, how='left', on='시군구명')
seoul_sgg.head()

In [ ]:
### 서울시 시군구 목록데이터에 인구수 병합
seoul_sgg = pd.merge(seoul_sgg, sgg_pop, how='inner', on='시군구명')
seoul_sgg.head()

In [ ]:
### 서울시 시군구 목록데이터에 통계데이터 병합
seoul_sgg = pd.merge(seoul_sgg, sgg_biz, how='inner', on='시군구명')
seoul_sgg.head()

In [ ]:
seoul_sgg.to_excel('data/seoul_sgg_stat.xlsx', index=False)